# A multivariate regression problem

In [1]:
import pandas
import numpy as np
import sklearn.linear_model as lm
from sklearn.model_selection import KFold
from sklearn import preprocessing as pre
from sklearn.decomposition import PCA
import random
import matplotlib.pyplot as plt

Load in the California housing dataset. Originally downloaded from https://github.com/ageron/handson-ml/tree/master/datasets/housing

In [2]:
housing = pandas.read_csv('data/housing.csv')

In [5]:
display(housing)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


This dataset has some special features which need some attention:
1. The variable ocean_proximity is a categorical variable
2. There are some missing values in the total_bedrooms column
Let us consider how to deal with each of these.

First, deal with the categorical variable ocean_proximity. How many distinct values does this take?

In [6]:
print(set(housing.ocean_proximity.values))
print(len(set(housing.ocean_proximity.values)))

{'NEAR OCEAN', 'NEAR BAY', 'ISLAND', '<1H OCEAN', 'INLAND'}
5


There are five unique values and so we will replace this with a one-hot vector of length 5 where 10000 corresponds to <1H OCEAN, 01000 is ISLAND etc. This is a standard way to represent categorical factors. The key is that for any entry (row) only one element of this vector should be 1 - hence one-hot.

In [3]:
housing['1h_ocean'] = [1 if i=='<1H OCEAN' else 0 for i in housing.ocean_proximity.values]
housing['island'] = [1 if i=='ISLAND' else 0 for i in housing.ocean_proximity.values]
housing['inland'] = [1 if i=='INLAND' else 0 for i in housing.ocean_proximity.values]
housing['near_ocean'] = [1 if i=='NEAR OCEAN' else 0 for i in housing.ocean_proximity.values]
housing['near_bay'] = [1 if i=='NEAR BAY' else 0 for i in housing.ocean_proximity.values]
housing.drop(columns=['ocean_proximity'], inplace=True)
display(housing)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,1h_ocean,island,inland,near_ocean,near_bay
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0,0,0,0,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0,0,0,0,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0,0,0,0,1
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0,0,0,0,1
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0,0,0,0,1
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,0,0,0,0,1
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,0,0,0,0,1
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,0,0,0,0,1
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,0,0,0,0,1
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,0,0,0,0,1


Now we need to look at the missing values in total_bedrooms, of which there are:

In [8]:
sum(housing.total_bedrooms.isna())

207

How can we deal with this? It depends. There are several strategies one can use.

* Replace with the average of the column, but this loses information about correlation
* Replace with the values from the nearest neighbour, based on the values of the other variables.
* Use some prior knowledge.

We will use some prior knowledge that it is highly likely that the number of bedrooms is strongly correlated with the total number of rooms. We'll fit a linear model to predict the missing values.

In [4]:
# Get the non-Nan indices
notna = housing.total_bedrooms.notna()

In [5]:
model = lm.LinearRegression()
model.fit(housing.total_rooms.values[notna].reshape(-1,1), housing.total_bedrooms.values[notna].reshape(-1,1))
model.score(housing.total_rooms.values[notna].reshape(-1,1), housing.total_bedrooms.values[notna].reshape(-1,1))

0.8656060227407114

This is a strong prediction so our intuition is correct. Now we predict the missing values.

In [6]:
isna = housing.total_bedrooms.isna()
missing_bedrooms = model.predict(housing.total_rooms.values[isna].reshape(-1,1))

Insert the imputed values into the table.

In [7]:
# Can ignore subsequent warning
housing.total_bedrooms.loc[isna] = np.squeeze(missing_bedrooms)

/home/jonwoodburn/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [62]:
display(housing)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,1h_ocean,island,inland,near_ocean,near_bay
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0,0,0,0,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0,0,0,0,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0,0,0,0,1
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0,0,0,0,1
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0,0,0,0,1
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,0,0,0,0,1
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,0,0,0,0,1
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,0,0,0,0,1
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,0,0,0,0,1
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,0,0,0,0,1


So we now have a complete dataset. Can we model it? An obvious thing to try first is a linear model, in which the house price is a weighted sum of the inputs; that is:
$$y = w_0 + w_1 x_1 + w_2 x_2 + w_M x_M$$
Let's assess this via cross-validation

In [9]:
house_price = housing["median_house_value"]
housing.drop("median_house_value", axis=1, inplace=True)
display(housing)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,1h_ocean,island,inland,near_ocean,near_bay
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,0,0,0,0,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,0,0,0,0,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,0,0,0,0,1
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,0,0,0,0,1
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,0,0,0,0,1
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,0,0,0,0,1
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,0,0,0,0,1
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,0,0,0,0,1
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,0,0,0,0,1
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,0,0,0,0,1


In [27]:
pca = PCA(n_components=8)
# X is the matrix transposed (n samples on the rows, m features on the columns)
pca.fit(housing)
new_df = pca.transform(housing)
new_df = pandas.DataFrame(new_df)
display(new_df)


,0,1,2,3,4,5,6,7
0,-2130.473680,-252.971952,63.948807,-6.109486,9.068669,2.661331,4.397033,-0.446819
1,4528.928179,-1035.431699,36.159318,-134.716163,0.170900,2.349832,1.710818,-0.000323
2,-1522.806232,-352.598637,108.071559,-5.841848,21.522648,2.591494,3.011889,-0.309462
3,-1653.615594,-204.861842,31.009076,-3.193302,20.790762,2.818190,1.968073,-0.113446
4,-1326.283582,-350.076732,27.140460,-8.933604,21.516503,2.885567,-0.034323,0.285889
5,-2034.634039,-178.667661,-17.295381,0.761225,19.935186,2.957564,0.699552,0.150125
6,-233.255393,-250.056042,-50.673804,-56.268738,22.239445,2.856858,-0.278172,0.162446
7,346.355714,-425.093863,-183.108260,-34.003456,23.411876,2.859155,-0.378276,0.213606
8,-128.400593,-142.713579,-207.774123,1.111963,12.593419,3.330658,-0.620231,0.496376
9,916.527335,-266.441546,-98.400723,-60.616030,24.091901,2.880961,-0.235139,0.107156


In [13]:
display(house_price)

0        452600.0
1        358500.0
2        352100.0
3        341300.0
4        342200.0
5        269700.0
6        299200.0
7        241400.0
8        226700.0
9        261100.0
10       281500.0
11       241800.0
12       213500.0
13       191300.0
14       159200.0
15       140000.0
16       152500.0
17       155500.0
18       158700.0
19       162900.0
20       147500.0
21       159800.0
22       113900.0
23        99700.0
24       132600.0
25       107500.0
26        93800.0
27       105500.0
28       108900.0
29       132000.0
           ...   
20610     45500.0
20611     47000.0
20612     48300.0
20613     53400.0
20614     58000.0
20615     57500.0
20616     55100.0
20617     70800.0
20618     63400.0
20619     99100.0
20620    100000.0
20621     77500.0
20622     67000.0
20623     65500.0
20624     87200.0
20625     72000.0
20626     93800.0
20627    162500.0
20628     92400.0
20629    108300.0
20630    112000.0
20631    107200.0
20632    115600.0
20633     98300.0
20634    1

In [28]:
house_price_df = pandas.DataFrame(house_price)
house_price_df.reset_index(drop=True, inplace=True)
new_df.reset_index(drop=True, inplace=True)
new_df["median_house_value"] = house_price_df
display(new_df)

,0,1,2,3,4,5,6,7,median_house_value
0,-2130.473680,-252.971952,63.948807,-6.109486,9.068669,2.661331,4.397033,-0.446819,452600.0
1,4528.928179,-1035.431699,36.159318,-134.716163,0.170900,2.349832,1.710818,-0.000323,358500.0
2,-1522.806232,-352.598637,108.071559,-5.841848,21.522648,2.591494,3.011889,-0.309462,352100.0
3,-1653.615594,-204.861842,31.009076,-3.193302,20.790762,2.818190,1.968073,-0.113446,341300.0
4,-1326.283582,-350.076732,27.140460,-8.933604,21.516503,2.885567,-0.034323,0.285889,342200.0
5,-2034.634039,-178.667661,-17.295381,0.761225,19.935186,2.957564,0.699552,0.150125,269700.0
6,-233.255393,-250.056042,-50.673804,-56.268738,22.239445,2.856858,-0.278172,0.162446,299200.0
7,346.355714,-425.093863,-183.108260,-34.003456,23.411876,2.859155,-0.378276,0.213606,241400.0
8,-128.400593,-142.713579,-207.774123,1.111963,12.593419,3.330658,-0.620231,0.496376,226700.0
9,916.527335,-266.441546,-98.400723,-60.616030,24.091901,2.880961,-0.235139,0.107156,261100.0


In [28]:
x = housing.values #returns a numpy array
min_max_scaler = pre.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pandas.DataFrame(x_scaled)
df.columns = housing.columns
display(df)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,1h_ocean,island,inland,near_ocean,near_bay
0,0.211155,0.567481,0.784314,0.022331,0.019863,0.008941,0.020556,0.539668,0.902266,0.0,0.0,0.0,0.0,1.0
1,0.212151,0.565356,0.392157,0.180503,0.171477,0.067210,0.186976,0.538027,0.708247,0.0,0.0,0.0,0.0,1.0
2,0.210159,0.564293,1.000000,0.037260,0.029330,0.013818,0.028943,0.466028,0.695051,0.0,0.0,0.0,0.0,1.0
3,0.209163,0.564293,1.000000,0.032352,0.036313,0.015555,0.035849,0.354699,0.672783,0.0,0.0,0.0,0.0,1.0
4,0.209163,0.564293,1.000000,0.041330,0.043296,0.015752,0.042427,0.230776,0.674638,0.0,0.0,0.0,0.0,1.0
5,0.209163,0.564293,1.000000,0.023323,0.032899,0.011491,0.031574,0.243921,0.525155,0.0,0.0,0.0,0.0,1.0
6,0.209163,0.563231,1.000000,0.064423,0.075729,0.030578,0.084361,0.217873,0.585979,0.0,0.0,0.0,0.0,1.0
7,0.209163,0.563231,1.000000,0.078895,0.106456,0.032344,0.106233,0.180694,0.466804,0.0,0.0,0.0,0.0,1.0
8,0.208167,0.563231,0.803922,0.064932,0.103042,0.033717,0.097681,0.108998,0.436495,0.0,0.0,0.0,0.0,1.0
9,0.209163,0.563231,1.000000,0.090213,0.109559,0.043387,0.117250,0.220087,0.507423,0.0,0.0,0.0,0.0,1.0


In [29]:
# First, extract the data into arrays
y = new_df.median_house_value.values.reshape(-1,1)
X = new_df.drop(columns=['median_house_value'], inplace=False).values
print(X.shape)
print(y.shape)
# Pull out 1000 values into a holdout set
holdout = random.sample(range(0,10640),1000)
X_holdout = X[holdout]
y_holdout = y[holdout]
Xt = np.delete(X, holdout, 0)
yt = np.delete(y, holdout, 0)
print(Xt.shape)
print(yt.shape)

(20640, 8)
(20640, 1)
(19640, 8)
(19640, 1)


In [30]:
Model = lm.LinearRegression()
# Have to shuffle the data because it is grouped.
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(Xt):
    X_train, X_test = Xt[train_index], Xt[test_index]
    y_train, y_test = yt[train_index], yt[test_index]
    Model.fit(X_train, y_train)
    print('Training error: ' + str(Model.score(X_train, y_train)))
    print('Testing error: ' + str(Model.score(X_test, y_test)))

Training error: 0.644633645637729
Testing error: 0.6412978402018965
Training error: 0.6419684924707715
Testing error: 0.6520394557081163
Training error: 0.6416560095855803
Testing error: 0.6532969397464443
Training error: 0.6506742361260623
Testing error: 0.6155730314600993
Training error: 0.6422176298628259
Testing error: 0.6503932114901725


Scores quoted are $R^2$ (coefficient of determination) values which range from 0 to 1. These are OK but there is much room for improvement and we ought to be able to do much better than this. Options that we could try are:

* Normalising/rescaling the data so that all variables have similar values?
* Expanding the basis to include terms that are non-linear in the variables?
* Removing redundant variables from the data - are there any that are correlated with each other?
* Regularisation?

This will be the task of the group assignment.